# 필수 모듈 가져오기

In [2]:
from numpy import int16
import requests
import json
from playsound import playsound
from scipy.io.wavfile import write
import sounddevice
import multiprocessing
from datetime import datetime
from bs4 import BeautifulSoup

# 카카오 텍스트 전송을 위한 토큰 발급

In [27]:
url = "https://kauth.kakao.com/oauth/token"
data = {
    "grant_type" : "authorization_code",
    "client_id" : "140bf781fe1ceef3dce75719d8dbffd0",
    "redirect_url" : "https://localhost.com",
    "code" : "M0N805Vx_nV8jEByycfj56YW8HvvIzOX-NtMisQmiIRgu_x7OLAgRfTcPItwzdNvNqa12gopdSkAAAF-eA5o0g",
}
res = requests.post(url, data=data)
tokens = res.json()
print(tokens)

{'access_token': 't9mSVJveGZkD8IVXZ9GO0Heq7m-2ktOlLJs4Swo9dBEAAAF-eBToeA', 'token_type': 'bearer', 'refresh_token': 'SrrNWR7H9Dse0nVkY2ZThMxsc_SbTXox2pTeQgo9dBEAAAF-eBTodw', 'expires_in': 21599, 'scope': 'talk_message', 'refresh_token_expires_in': 5183999}


# 음성 녹음

In [31]:
fs=16000
second=5
print("recording....")
record_voice=sounddevice.rec(int(second * fs), samplerate=fs, channels=1, dtype=int16)
sounddevice.wait()
write('temp.wav', fs, record_voice)

recording....


# STT 서비스 요청

In [32]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"
key = "140bf781fe1ceef3dce75719d8dbffd0"
headers = {
    "Content-Type": "application/octet-stream",
    "Authorization": "KakaoAK " + key,
}
with open('temp.wav', 'rb') as fp:
    audio = fp.read()
res = requests.post(url, headers=headers, data=audio)

# 요청 결과 출력

In [33]:
json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
result = json.loads(json_string)['value']
print(result)

메모해줘


# 키워드 비교

In [34]:
keyword = ''
result = result.replace(" ","")
if "메모" in result:
    keyword = 'MEMO'
else:
    keyword = 'UNKNOWN'

# 키워드 출력

In [35]:
print(keyword)

MEMO


# 키워드가 MEMO라면 메모 요청을 data로 설정,
# 그렇지 않다면 data로 무슨 말인지 잘 모르겠어요 로 설정

In [36]:
data = ''
if keyword == 'MEMO':
    data = f'<speak>네... 메모할 내용을 말씀하세요.</speak>'
else:
    data = f'<speak>무슨 말인지 잘 모르겠어요</speak>'
data = data.encode('utf-8').decode('latin1')    

# TTS 서비스 요청

In [37]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"
key = "140bf781fe1ceef3dce75719d8dbffd0"
headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + key,
}
res = requests.post(url, headers=headers, data=data)
if (res.status_code != 200):
    print(f'TTS 요청이 실패하였습니다.(code={res.status_code})')
else:
    with open('result.mp3', "wb") as f:
        f.write(res.content)

# TTS 서비스 결과 출력

In [38]:
p = multiprocessing.Process(target=playsound, args=('result.mp3',))
p.start()
input("press ENTER to stop playback")
p.terminate()

press ENTER to stop playback


# 음성 녹음

In [39]:
fs=16000
second=5
print("recording....")
record_voice=sounddevice.rec(int(second * fs), samplerate=fs, channels=1, dtype=int16)
sounddevice.wait()
write('temp.wav', fs, record_voice)

recording....


# STT 요청

In [40]:
url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"
key = "140bf781fe1ceef3dce75719d8dbffd0"
headers = {
    "Content-Type": "application/octet-stream",
    "Authorization": "KakaoAK " + key,
}
with open('temp.wav', 'rb') as fp:
    audio = fp.read()
res = requests.post(url, headers=headers, data=audio)

# 요청 결과 출력

In [41]:
json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}')+1]
result = json.loads(json_string)['value']
print(result)

오후 5시까지 사업 계획서 제출


# 카카오톡 나와의 대화로 전송

In [42]:
url = "https://kapi.kakao.com/v2/api/talk/memo/default/send"
key = "kB5lN6uA-0kEoHbxiaGy48215f0UXJ-D3BJGrwo9c-wAAAF-d_mTlA"
headers = {
    "Authorization": "Bearer " + key
}
data = {
    "template_object" : json.dumps({ 
        "object_type" : "text",
        "text" : result,
        "link" : {
            "web_url" : "www.naver.com"
        }
    })
}

response = requests.post(url, headers=headers, data=data)
print(response.status_code)
if response.json().get('result_code') == 0:
    print('메시지를 성공적으로 보냈습니다.')
else:
    print('메시지를 성공적으로 보내지 못했습니다. 오류메시지 : ' + str(response.json()))

200
메시지를 성공적으로 보냈습니다.
